In [ ]:
!pip install pymorphy3
!pip uninstall -y torch torchvision transformers
!pip install datasets torch torchvision transformers scikit-learn imbalanced-learn
!pip install evaluate rouge-score sacrebleu bert-score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.1 MB/s eta 0:00:00
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: transformers 4.51.1
Uninstalling transformers-4.51.1:
  Successfully uninstalled transformers-4.51.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
import evaluate
import gc
import nltk
import numpy as np
import os
import pandas as pd
import plotly.express as px
import pymorphy3
import random
import re
import textwrap
import torch
import torch.nn as nn
import transformers
import warnings
from bert_score import score as bert_score
from datasets import Dataset, concatenate_datasets, IterableDataset, load_dataset
from google.colab import drive
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import (AutoModelForSequenceClassification,
                         AutoTokenizer,
                         DataCollatorForLanguageModeling,
                         DataCollatorForSeq2Seq,
                         get_cosine_schedule_with_warmup,
                         MBart50Tokenizer,
                         MBart50TokenizerFast,
                         MBartForConditionalGeneration,
                         TrainingArguments,
                         Trainer
)
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50Tokenizer.from_pretrained(model_path)

In [ ]:
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
texts = ["Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили заранее, далее заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда полы сделаны, а стены еще нет. Мы попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили ответ, что установка 1 двери для мастера неинтересный выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно, но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые отделочные работы. После чего был повторный выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по повышенном тарифу и за то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделывать? А главное за что я заплатила 22,5 руб. за установку чего, кривизны? Офис никаких комментариев не давал, поставили криво и забыли, крайне неприятная ситуация. Ответили только после того, как я написала отрицательный отзыв. Это не обычные двери, а те, что монтируются в стену здесь доборами и планками ничего не исправить, нужно ломать и строить заново! Ванная - там вообще керамогранит уже на стенах!Безответственные сотрудники! Заказывали в салоне Коммунарка/Бутово, крайне не рекомендую связываться с данной компанией!\nДополню свой и без того отрицательный отзыв, еще одним минусом сотрудничества с данной компанией. Устанавливая ручки и замки на двери, мы поняли, что на обратной двери (она больше стандартных) неподходящей длины четырехгранник, на что в офисе мне ответили, что его можно купить в магазине и в комплектацию он не входит, хорошо пену покупать не нужно и петли дверные. И более того, побродив по просторам интернета, мы поняли, что этих четырехгранников нигде нет, о чем я снова написала сотрудникам магазина, после чего получила ответ, что у них их не было и вот они появились, могут нам продать! Продать, то что входит в комплект! После того, как я подготовила досудебную претензию, магазин все же решил довезти недостающий четырехгранник. Вывод, чего-то добиться можно, но со скандалами и тратой нервов..", "Каждый раз, посещая этот магазин, чувствую неприятную атмосферу, из-за нервных кассиров, был свидетелем, когда кассир нагрубила пожилой женщине, потребовав у нее быстрее расплатиться так-как она задерживает очередь, она терпеливо с дрожащими руками расплатилась, после этого осадок остался не приятный. Интересно Чижик набирает кассиров именно с качествами нервно больных людей? Ужасный персонал!\n", "Клиника не дорожит клиентами! Ни о какой конфиденциальности не может быть и речи! Всегда нужно ожидать, что ваши фото с процедур появятся на страницах соцсетей, даже если вы категорически против и подписали договор о неразглашении! Записываешься на процедуру к одному сотруднику а делает другой, хотя ты подстраиваешься именно под определенного косметолога! Отвратительно! О минусах можно писать бесконечно. Я очень разочарована этой клиникой и главным косметологом Лолой!", "Кроме того, что место красивое и необычное, плюсов нет. Были 1 июня.\nАренда 2500 час, но это просто за иглу. 1000 пробковый сбор, хотя по умолчанию в заведении нет напитков и еды, все приносишь с собой. Сервировка стола платная, при этом сюда входит тарелка,стакан и приборы (Рюмка или бокал за отдельную плату). Посуда самая дешевая, при этом за 1 разбитую единицу штраф-1000. Стол в иглу, которое досталось нам был не закреплен. Т.е крышка и подножья стола существовали отдельно, каждый раз стол болтался, и не разбить там посуду было сложно. Музыкальная колонка тоже попалась «уставшая» громкость не регулировалась. Заказ был на 3 часа+ 1 в подарок, при этом девушка-администратор, насчитала за 4 часа. Прям настаивать пришлось, чтобы сделать перерасчет. Ощущение, что за все пытаются сбить деньги, при этом сервис на 2 балла.", "Самый отвратительный сервис, который можно придумать.\n\nПостоянно сдвигают сроки доставки. В магазин невозможно дозвониться. Не берут трубку.\nНа 15 звонков они отвечают сообщением в вотс ап.\n\nНа просьбу, чтобы связались по телефону сказали 'Да, вам сегодня позвонят' никто так и не перезвонил. \n\n\nДоставка должна была быть в четверг. После моих 10 звонков написали, что в пятницу.\n\nВ пятницу после моих 10 звонков написали, что в воскресенье.\n\nВ воскресенье после того как я вновь сама 10 раз позвонила сказали, что уже приедет сегодня так как это передано в службу доставки.\n\nНикто не приехал).\n\nДеньги, конечно же, сняли сразу за заказ). А вот за что сняли, хз) видимо просто так. \n\nМаксимум, что они пишут на все это: приносим наши извинения....принесите мне лучше заказ, а извинения..думаю разберетесь куда деть;).\n\nВ общем, если вы хотите заказать подарок к определенному сроку - не рекомендую. \nЕсли вы хотите потратить в пустую деньги и нервы - идеальное место)).", "Если Вам нужны качественные услуги, то однозначно не сюда. Ребята просто зарабатывают деньги и ничего личного. За качество выполняемой работы и используемых материалов не отвечают от слова совсем. И если Вы предъявите претензии к качеству работы, получите в ответ один негатив и будете сами виноваты, что заказали услугу из некачественных материалов. Заказывала у них цветную печать А2 плакатов более 10 лет на простой белой бумаге. Качество было нормальное. Последний раз получила плакат за 400 рублей с искажёнными и изудорованными до неузнаваемости лицами. При этом печать абсолютно не соответствует качеству моего файла. На мой вопрос, что это такое, мне ответили: А зачем Вы заказывали печать на дешёвой бумаге? То есть за 400 рублей я получила кусочек бумаги, который просто нужно отправить в мусор. Пришлось подарить им плакат на память. Пусть любуются своей работой. \nА мои 400 рублей переведите в фонд СВО. Вы их не заработали! Халтура, да и только.", "Всем добрый день! Не могу сказать насчет квалификации специалистов, но организация работы данного заведения это полное дно.\nМне нужно было попасть на прием к врачу специалисту, но, как оказалось, талон к врачу специалисту, выдавал только врач общей практики или врач терапевт. Ладно, подумал я, и решил записаться к врачу общей практики. Для этого 29.03.23 я взял талон на ближайший свободный день 7.04.23 ( ожидание 8 рабочих дней) к врачу общей практики. Врач общей практики перед тем как записать к врачу специалисту, выдал мне направление на анализы (самые простые кровь , глюкоза и т.д) на 20.04.23 - ожидание 9 рабочих дней. После этого я должен был ждать результатов анализа 2-4 рабочих дня, потом записаться повторно к врачу общей практики с результатами анализов (5-10 рабочих дней) и наконец попасть к врачу специалисту - ожидание приблизительно еще 14 рабочих дней. Итого - общее время ожидания 45 рабочих дней. Больше ДВУХ месяцев !!! Это как нужно умудриться построить работу медицинского учреждения чтобы достичь такого времени ожидания??\nРазумеется, чтобы сократить время, я попробовал сдать анализы на следующий день. Но мне отказали, мотивируя это тем что я пришел не в свой день . Хотя очереди не было, а в кабинете на приеме анализов работало три человека и кто то из них явно мог меня принять. Пришлось ехать сдавать анализы в хейликс и записываться на прием в платную клинику. \nПС. Интересный факт, что в коммерческих лабораториях , на приеме анализов работает ОДИН человек и как правило нет очередей. Поэтому что то мне подсказывает, что очереди в поликлиниках это результат работы администрации."]
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    forced_bos_id = tokenizer.lang_code_to_id["ru_RU"]
    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        forced_bos_token_id=forced_bos_id,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    wrapped_text = textwrap.fill(text, width=120, break_long_words=False)
    wrapped_summary = textwrap.fill(summary, width=120, break_long_words=False)

    print(f"Длина отзыва: {len(text)}; Отзыв: {wrapped_text}")
    print(f"Длинна суммаризации: {len(summary)}; Суммаризация отзыва:{wrapped_summary}")
    print()

Длина отзыва: 2260; Отзыв: Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили
заранее, далее заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда
полы сделаны, а стены еще нет. Мы попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили
ответ, что установка 1 двери для мастера неинтересный выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно,
но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые отделочные работы. После чего был повторный
выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по повышенном тарифу и за
то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не
отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделывать? А
главное за что я заплатила 22,5 руб

In [ ]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
texts = ["Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили заранее, далее заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда полы сделаны, а стены еще нет. Мы попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили ответ, что установка 1 двери для мастера неинтересный выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно, но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые отделочные работы. После чего был повторный выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по повышенном тарифу и за то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделывать? А главное за что я заплатила 22,5 руб. за установку чего, кривизны? Офис никаких комментариев не давал, поставили криво и забыли, крайне неприятная ситуация. Ответили только после того, как я написала отрицательный отзыв. Это не обычные двери, а те, что монтируются в стену здесь доборами и планками ничего не исправить, нужно ломать и строить заново! Ванная - там вообще керамогранит уже на стенах!Безответственные сотрудники! Заказывали в салоне Коммунарка/Бутово, крайне не рекомендую связываться с данной компанией!\nДополню свой и без того отрицательный отзыв, еще одним минусом сотрудничества с данной компанией. Устанавливая ручки и замки на двери, мы поняли, что на обратной двери (она больше стандартных) неподходящей длины четырехгранник, на что в офисе мне ответили, что его можно купить в магазине и в комплектацию он не входит, хорошо пену покупать не нужно и петли дверные. И более того, побродив по просторам интернета, мы поняли, что этих четырехгранников нигде нет, о чем я снова написала сотрудникам магазина, после чего получила ответ, что у них их не было и вот они появились, могут нам продать! Продать, то что входит в комплект! После того, как я подготовила досудебную претензию, магазин все же решил довезти недостающий четырехгранник. Вывод, чего-то добиться можно, но со скандалами и тратой нервов..", "Каждый раз, посещая этот магазин, чувствую неприятную атмосферу, из-за нервных кассиров, был свидетелем, когда кассир нагрубила пожилой женщине, потребовав у нее быстрее расплатиться так-как она задерживает очередь, она терпеливо с дрожащими руками расплатилась, после этого осадок остался не приятный. Интересно Чижик набирает кассиров именно с качествами нервно больных людей? Ужасный персонал!\n", "Клиника не дорожит клиентами! Ни о какой конфиденциальности не может быть и речи! Всегда нужно ожидать, что ваши фото с процедур появятся на страницах соцсетей, даже если вы категорически против и подписали договор о неразглашении! Записываешься на процедуру к одному сотруднику а делает другой, хотя ты подстраиваешься именно под определенного косметолога! Отвратительно! О минусах можно писать бесконечно. Я очень разочарована этой клиникой и главным косметологом Лолой!", "Кроме того, что место красивое и необычное, плюсов нет. Были 1 июня.\nАренда 2500 час, но это просто за иглу. 1000 пробковый сбор, хотя по умолчанию в заведении нет напитков и еды, все приносишь с собой. Сервировка стола платная, при этом сюда входит тарелка,стакан и приборы (Рюмка или бокал за отдельную плату). Посуда самая дешевая, при этом за 1 разбитую единицу штраф-1000. Стол в иглу, которое досталось нам был не закреплен. Т.е крышка и подножья стола существовали отдельно, каждый раз стол болтался, и не разбить там посуду было сложно. Музыкальная колонка тоже попалась «уставшая» громкость не регулировалась. Заказ был на 3 часа+ 1 в подарок, при этом девушка-администратор, насчитала за 4 часа. Прям настаивать пришлось, чтобы сделать перерасчет. Ощущение, что за все пытаются сбить деньги, при этом сервис на 2 балла.", "Самый отвратительный сервис, который можно придумать.\n\nПостоянно сдвигают сроки доставки. В магазин невозможно дозвониться. Не берут трубку.\nНа 15 звонков они отвечают сообщением в вотс ап.\n\nНа просьбу, чтобы связались по телефону сказали 'Да, вам сегодня позвонят' никто так и не перезвонил. \n\n\nДоставка должна была быть в четверг. После моих 10 звонков написали, что в пятницу.\n\nВ пятницу после моих 10 звонков написали, что в воскресенье.\n\nВ воскресенье после того как я вновь сама 10 раз позвонила сказали, что уже приедет сегодня так как это передано в службу доставки.\n\nНикто не приехал).\n\nДеньги, конечно же, сняли сразу за заказ). А вот за что сняли, хз) видимо просто так. \n\nМаксимум, что они пишут на все это: приносим наши извинения....принесите мне лучше заказ, а извинения..думаю разберетесь куда деть;).\n\nВ общем, если вы хотите заказать подарок к определенному сроку - не рекомендую. \nЕсли вы хотите потратить в пустую деньги и нервы - идеальное место)).", "Если Вам нужны качественные услуги, то однозначно не сюда. Ребята просто зарабатывают деньги и ничего личного. За качество выполняемой работы и используемых материалов не отвечают от слова совсем. И если Вы предъявите претензии к качеству работы, получите в ответ один негатив и будете сами виноваты, что заказали услугу из некачественных материалов. Заказывала у них цветную печать А2 плакатов более 10 лет на простой белой бумаге. Качество было нормальное. Последний раз получила плакат за 400 рублей с искажёнными и изудорованными до неузнаваемости лицами. При этом печать абсолютно не соответствует качеству моего файла. На мой вопрос, что это такое, мне ответили: А зачем Вы заказывали печать на дешёвой бумаге? То есть за 400 рублей я получила кусочек бумаги, который просто нужно отправить в мусор. Пришлось подарить им плакат на память. Пусть любуются своей работой. \nА мои 400 рублей переведите в фонд СВО. Вы их не заработали! Халтура, да и только.", "Всем добрый день! Не могу сказать насчет квалификации специалистов, но организация работы данного заведения это полное дно.\nМне нужно было попасть на прием к врачу специалисту, но, как оказалось, талон к врачу специалисту, выдавал только врач общей практики или врач терапевт. Ладно, подумал я, и решил записаться к врачу общей практики. Для этого 29.03.23 я взял талон на ближайший свободный день 7.04.23 ( ожидание 8 рабочих дней) к врачу общей практики. Врач общей практики перед тем как записать к врачу специалисту, выдал мне направление на анализы (самые простые кровь , глюкоза и т.д) на 20.04.23 - ожидание 9 рабочих дней. После этого я должен был ждать результатов анализа 2-4 рабочих дня, потом записаться повторно к врачу общей практики с результатами анализов (5-10 рабочих дней) и наконец попасть к врачу специалисту - ожидание приблизительно еще 14 рабочих дней. Итого - общее время ожидания 45 рабочих дней. Больше ДВУХ месяцев !!! Это как нужно умудриться построить работу медицинского учреждения чтобы достичь такого времени ожидания??\nРазумеется, чтобы сократить время, я попробовал сдать анализы на следующий день. Но мне отказали, мотивируя это тем что я пришел не в свой день . Хотя очереди не было, а в кабинете на приеме анализов работало три человека и кто то из них явно мог меня принять. Пришлось ехать сдавать анализы в хейликс и записываться на прием в платную клинику. \nПС. Интересный факт, что в коммерческих лабораториях , на приеме анализов работает ОДИН человек и как правило нет очередей. Поэтому что то мне подсказывает, что очереди в поликлиниках это результат работы администрации."]
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    forced_bos_id = tokenizer.lang_code_to_id["ru_RU"]
    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        forced_bos_token_id=forced_bos_id,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    wrapped_text = textwrap.fill(text, width=120, break_long_words=False)
    wrapped_summary = textwrap.fill(summary, width=120, break_long_words=False)

    print(f"Длина отзыва: {len(text)}; Отзыв: {wrapped_text}")
    print(f"Длинна суммаризации: {len(summary)}; Суммаризация отзыва:{wrapped_summary}")
    print()

Длина отзыва: 2260; Отзыв: Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили
заранее, далее заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда
полы сделаны, а стены еще нет. Мы попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили
ответ, что установка 1 двери для мастера неинтересный выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно,
но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые отделочные работы. После чего был повторный
выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по повышенном тарифу и за
то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не
отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделывать? А
главное за что я заплатила 22,5 руб

In [ ]:
dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
model_path = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50Tokenizer.from_pretrained(model_path)

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
references = []
predictions = []
sources = []

for sample in tqdm(test_dataset):
    text = sample["text"]
    reference = sample["summary"]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=128,
            num_beams=4,
            forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
        )
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(reference)
    sources.append(text)
# ========== BERTScore ==========
P, R, F1_bert = bert_score(predictions, references, lang="ru", verbose=True)

# ========== F1 по ключевым словам ==========
nltk.download("punkt")
nltk.download("stopwords")

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

def extract_keywords(text):
    tokens = word_tokenize(text.lower())
    lemmas = [morph.parse(w)[0].normal_form for w in tokens if w.isalpha() and w not in stop_words]
    return set(lemmas)

f1_scores = []
for ref, pred in zip(references, predictions):
    ref_kw = extract_keywords(ref)
    pred_kw = extract_keywords(pred)
    all_kw = list(ref_kw.union(pred_kw))
    ref_vec = [1 if w in ref_kw else 0 for w in all_kw]
    pred_vec = [1 if w in pred_kw else 0 for w in all_kw]
    if sum(ref_vec) > 0 or sum(pred_vec) > 0:
        f1_scores.append(f1_score(ref_vec, pred_vec))

f1_keyword = sum(f1_scores) / len(f1_scores)

# ========== ROUGE-L ==========
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_f1 = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, predictions)]

In [ ]:
print(f"\033[1mBERTScore F1: {F1_bert.mean().item()}")
print(f"F1 по ключевым словам: {f1_keyword}")
print(f"ROUGE-L: {sum(rouge_l_f1)/len(rouge_l_f1)}")

BERTScore F1: 0.7234094142913818
F1 по ключевым словам: 0.36163304119813117
ROUGE-L: 0.19094444825062734


In [ ]:
model_path = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50Tokenizer.from_pretrained(model_path)
references = []
predictions = []
sources = []

for sample in tqdm(test_dataset):
    text = sample["text"]
    reference = sample["summary"]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=128,
            num_beams=4,
            forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
        )
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(reference)
    sources.append(text)
# ========== BERTScore ==========
P, R, F1_bert = bert_score(predictions, references, lang="ru", verbose=True)

# ========== F1 по ключевым словам ==========
nltk.download("punkt")
nltk.download("stopwords")

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

def extract_keywords(text):
    tokens = word_tokenize(text.lower())
    lemmas = [morph.parse(w)[0].normal_form for w in tokens if w.isalpha() and w not in stop_words]
    return set(lemmas)

f1_scores = []
for ref, pred in zip(references, predictions):
    ref_kw = extract_keywords(ref)
    pred_kw = extract_keywords(pred)
    all_kw = list(ref_kw.union(pred_kw))
    ref_vec = [1 if w in ref_kw else 0 for w in all_kw]
    pred_vec = [1 if w in pred_kw else 0 for w in all_kw]
    if sum(ref_vec) > 0 or sum(pred_vec) > 0:
        f1_scores.append(f1_score(ref_vec, pred_vec))

f1_keyword = sum(f1_scores) / len(f1_scores)

# ========== ROUGE-L ==========
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_f1 = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, predictions)]

In [ ]:
print(f"\033[1mBERTScore F1: {F1_bert.mean().item()}")
print(f"F1 по ключевым словам: {f1_keyword}")
print(f"ROUGE-L: {sum(rouge_l_f1)/len(rouge_l_f1)}")

BERTScore F1: 0.6841965317726135
F1 по ключевым словам: 0.2204382677033209
ROUGE-L: 0.09034048506074986


In [ ]:
df = pd.DataFrame({
    'Метрика': ['Accuracy', 'F1', 'Precision', 'Recall'],
    'Значение': [0.9641428571428572, 0.9640761414054673, 0.9698784772216783, 0.958342818119736],
    'Модель': ['Дообученная классификационная модель BART'] * 4
})
fig = px.bar(df, x='Метрика', y='Значение', barmode='group', title='Метрики дообученной классификационной модели', text='Значение')
fig.update_traces(
    width=0.25,
    texttemplate='%{text:.6f}',
    textposition='inside'
)

fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)
fig.show()

In [ ]:
df = pd.DataFrame({
    'Метрика': ['BERTScore', 'F1', 'ROUGE-L'] * 2,
    'Значение': [0.6841965317726135, 0.2204382677033209, 0.09034048506074986, 0.7234094142913818, 0.36163304119813117, 0.19094444825062734],
    'Модель': ['Дообученная генеративная модель BART'] * 3 + ['Оригинальная генеративная модель BART'] * 3
})

fig = px.bar(
    df,
    x='Метрика',
    y='Значение',
    color='Модель',
    barmode='group',
    title='Сравнение метрик двух генеративных моделей',
    text=df['Значение']
)

fig.update_traces(
    width=0.25,
    texttemplate='%{text:.6f}',
    textposition='inside'
)

fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)
fig.show()

In [ ]:
df = pd.DataFrame({
    'Метрика': ['BERTScore', 'F1', 'ROUGE-L'],
    'Значение': [0.6841965317726135, 0.2204382677033209, 0.09034048506074986],
    'Модель': ['Дообученная генеративная модель BART'] * 3
})

fig = px.bar(
    df,
    x='Метрика',
    y='Значение',
    color='Модель',
    barmode='group',
    title='Метрики дообученной генеративной модели',
    text=df['Значение']
)

fig.update_traces(
    width=0.25,
    texttemplate='%{text:.6f}',
    textposition='inside'
)

fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)
fig.show()